# Dataset - Generative Dog Images

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1tKOTjbtdxS-pzw80JHN-2FUiZA7htL7p' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1tKOTjbtdxS-pzw80JHN-2FUiZA7htL7p" -O data.zip && rm -rf /tmp/cookies.txt

--2021-08-25 13:46:26--  https://docs.google.com/uc?export=download&confirm=UchH&id=1tKOTjbtdxS-pzw80JHN-2FUiZA7htL7p
Resolving docs.google.com (docs.google.com)... 142.251.2.138, 142.251.2.101, 142.251.2.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-84-docs.googleusercontent.com/docs/securesc/q855d61ksk3m8tv96m4m8sfhkg46l2vh/bo2s7uh60o62ia0qkhaof5q96s0ev3d6/1629899175000/06049624160957438625/15222733518293580913Z/1tKOTjbtdxS-pzw80JHN-2FUiZA7htL7p?e=download [following]
--2021-08-25 13:46:26--  https://doc-04-84-docs.googleusercontent.com/docs/securesc/q855d61ksk3m8tv96m4m8sfhkg46l2vh/bo2s7uh60o62ia0qkhaof5q96s0ev3d6/1629899175000/06049624160957438625/15222733518293580913Z/1tKOTjbtdxS-pzw80JHN-2FUiZA7htL7p?e=download
Resolving doc-04-84-docs.googleusercontent.com (doc-04-84-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to

In [ ]:
import os

!mkdir '../input'
print(os.listdir("../input"))

[]


In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: Annotation.zip          
  inflating: all-dogs.zip            


In [ ]:
import zipfile

!mkdir '../input/annotation'

with zipfile.ZipFile("Annotation.zip","r") as zip_ref:
    zip_ref.extractall("../input/annotation")

In [ ]:
!mkdir '../input/all-dogs'

with zipfile.ZipFile("all-dogs.zip","r") as zip_ref:
    zip_ref.extractall("../input/all-dogs")

# Generative Adversarial Network 
# Šta je GAN?

GAN je generativna kontradiktorna mreža - generativni model koji u sebi koristi modele generator i diskriminator kako bi generisao nove fotografije.
Ova dva modela međusobno utiču jedan na drugi.
Generisaće se nove fotografije u odnosu na distribuciju iz podataka dataseta.

# Kako funkcioniše GAN?
Prvenstveno je potrebno odabrati dataset (slike pasa), diskriminatoru dati podatke da ih prouči (kako bi mogao odrediti šta je pravi pas), ovde generator kao ulaz uzima random vektor (buka, skup brojeva) i generiše nove fotografije, diskriminator sad treba da odluči da li je fotografija prava ili lažna, tj. generisana.
Na izlazu se dobijaju vrednosti pomoću kojih se optimizuje rad oba modela.
Generator će generisati bolje fotografije, a diskriminator će davati bolju prognozu da li je fotografija prava ili ne.
Ovaj proces se dešava iterativno kroz određeni broj epoha treniranja modela.

In [ ]:
import numpy as np 
import pandas as pd

import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)


   * dataroot - putanja do dataset foldera.
   * workers - broj worker thread-ova za učitavanje podataka preko DataLoader-a
   * batch_size - veličina skupa koji će se koristiti u treniranju modela. Istraživanja na DCGAN temu pokazuju da je najbolje koristiti 128
   * image_size - veličina fotografije koja se koristi u treniranju. Implementacija je urađena za fotografije veličine 64x64. Ukoliko želimo drugu veličinu, implementacija D i G se mora menjati.
   * nc - broj kanala za boje u fotografijama. Za fotografije u boji ovo će biti 3
   * nz - length of latent vector
   * ngf - relates to the depth of feature maps carried through the generator
   * ndf - sets the depth of feature maps propagated through the discriminator
   * num_epochs - broj epoha za treniranje. Duže treniranje će dati bolje rezultate, ali će i dosta duže trajati
   * lr - learning rate for training. As described in the DCGAN paper, this number should be 0.0002
   * beta1 - beta1 hyperparameter for Adam optimizers. As described in paper, this number should be 0.5
   * ngpu - number of GPUs available. If this is 0, code will run in CPU mode. If this number is greater than 0 it will run on that number of GPUs


In [ ]:
dataroot = '../input/all-dogs'
workers = 2
batch_size = 128
image_size = 64
nc = 3
nz = 100
ngf = 64
ndf = 64
num_epochs = 300
lr = 0.0002
beta1 = 0.5
ngpu = 1
ComputeLB = False
DogsOnly = True

In [ ]:
import numpy as np, pandas as pd, os
import xml.etree.ElementTree as ET 
import matplotlib.pyplot as plt, zipfile 
from PIL import Image 
from glob import glob

ROOT = '../input/generative-dog-images/'
if not ComputeLB: ROOT = '../input/'
IMAGES = os.listdir(ROOT + 'all-dogs/all-dogs/')
breeds = os.listdir(ROOT + 'annotation/Annotation/') 

idxIn = 0; namesIn = []
imagesIn = np.zeros((25000,64,64,3))

# iseci fotografije uz pomoć anotacija
# https://www.kaggle.com/paulorzp/show-annotations-and-breeds
if DogsOnly:
    for breed in breeds:
        for dog in os.listdir(ROOT+'annotation/Annotation/'+breed):
            try: img = Image.open(ROOT+'all-dogs/all-dogs/'+dog+'.jpg') 
            except: continue           
            tree = ET.parse(ROOT+'annotation/Annotation/'+breed+'/'+dog)
            root = tree.getroot()
            objects = root.findall('object')
            for o in objects:
                bndbox = o.find('bndbox') 
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                w = np.min((xmax - xmin, ymax - ymin))
                img2 = img.crop((xmin, ymin, xmin+w, ymin+w))
                img2 = img2.resize((64,64), Image.ANTIALIAS)
                imagesIn[idxIn,:,:,:] = np.asarray(img2)
                #if idxIn%1000==0: print(idxIn)
                namesIn.append(breed)
                idxIn += 1
    idx = np.arange(idxIn)
    np.random.shuffle(idx)
    imagesIn = imagesIn[idx,:,:,:]
    namesIn = np.array(namesIn)[idx]
    
# random iseci sve fotografije
else:
    x = np.random.choice(np.arange(20579),10000)
    for k in range(len(x)):
        img = Image.open(ROOT + 'all-dogs/all-dogs/' + IMAGES[x[k]])
        w = img.size[0]
        h = img.size[1]
        sz = np.min((w,h))
        a=0; b=0
        if w<h: b = (h-sz)//2
        else: a = (w-sz)//2
        img = img.crop((0+a, 0+b, sz+a, sz+b))  
        img = img.resize((64,64), Image.ANTIALIAS)   
        imagesIn[idxIn,:,:,:] = np.asarray(img)
        namesIn.append(IMAGES[x[k]])
        if idxIn%1000==0: print(idxIn)
        idxIn += 1
    
# prikaz isečenih slika
x = np.random.randint(0,idxIn,25)
for k in range(5):
    plt.figure(figsize=(15,3))
    for j in range(5):
        plt.subplot(1,5,j+1)
        img = Image.fromarray( imagesIn[x[k*5+j],:,:,:].astype('uint8') )
        plt.axis('off')
        if not DogsOnly: plt.title(namesIn[x[k*5+j]],fontsize=11)
        else: plt.title(namesIn[x[k*5+j]].split('-')[1],fontsize=11)
        plt.imshow(img)
    plt.show()

In [ ]:
# učitavanje dataseta
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)


device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

# Implementacija

Nakon definisanja parametara i učitavanja dataseta, može se početi sa implementacijom. Počećemo sa strategijom inicijalizacije težina, zatim ćemo proći kroz generator, diskriminator, funkcije gubitka i treniranje modela.


# Inicijalizacija težina

U DCGAN istraživanju, 
autori su zaključili da sve težine modela treba da budu random inicijalizovane iz normalne distribucije sa parametrima mean-0, stdev-0.02.
Funkcija weights_init uzima inicijalizovani model kao ulaz i reinicijalizuje sve konvolucijske, konvoluciono-transpozicione i batch normalizacijske slojeve da zadovolji kriterijume. Ova funkcija se primenjuje na modele odmah nakon inicijalizacije.

In [ ]:
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv') != -1:
    nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif classname.find('BatchNorm') != -1:
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.normal_(m.bias.data, 0)

# Generator

Generator, G,
 je dizajniran da mapira latentni vektor prostora (z) u prostor podataka. Pošto su naši podaci fotografije, konvertovanje z u prostor podataka znači kreirati RGB fotografiju istih dimenzija kao naši trening podaci (npr. 3x64x64).

U praksi, ovo je postignuto pomoću serije dvodimenzionih konvolucijskih transpozicionih slojeva, svaki uparen sa 2D batch normalizacionim slojem i ReLu aktivacijom.

Izlaz iz generatora se pušta kroz tanh funkciju i vraća kao vrednost između [-1, 1].

Vrlo je bitno da postoje batch norm. funkcije posle conv-transpore slojeva (Po istraživanju o DCGAN-ovima).
Ovi slojevi pomažu flow-u gradijenata tokom treniranja.

Na fotografiji ispod je prikazan DCGAN:

<img src="https://pytorch.org/tutorials/_images/dcgan_generator.png" width=800>

In [ ]:
# definisanje generatora (2d conv-transpose -> batch norm -> relu -> 4 convolutions)
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
netG = Generator(ngpu).to(device)
netG.apply(weights_init)
print(netG)

# Diskriminator

Diskriminator, D,
je binarna klasifikaciona mreža koja uzima fotografiju kao ulaz i na izlazu daje verovatnoću da je ulazna fotografija prava (da nije generisana).
 Ovde, D uzima 3x64x64 ulaznu fotografiju, procesuira je kroz seriju 2D konvolucije, 2D batch normalizacije, i LeakyReLU slojeva, i na izlazu daje verovatnoću uz pomoć Sigmoid aktivacione funkcije.
 Ova arhitektura može biti proširena sa još slojeva ukoliko je to potrebno, iako ne bi bilo korisno u Conv2d, batch norm. ili LeakyReLU.

**DCGAN istraživanje spominje da je dobra praksa koristiti 2d konvoluciju umesto pooling-a za downsampl-ovanje fotografija jer omogućava da mreža uči iz svoje pooling funkcije.**

**Takođe, batch normalizacija i leaky relu funkcije promovišu bolji gradient flow što je ključno za proces učenja i G i D.**

In [ ]:
# definisanje diskriminatora (2d conv - batch norm - leaky relu - 4 konvolucije - sigmoid funkcija za verovatnoću)
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
print(netD)

# Funkcija gubitka i optimizatori

Kada su definisani G i D,
možemo specifirati kako će učiti kroz funkcije gubitka i optimizatore. Koristićemo **binary cross entropy loss funkciju (BCELoss)** koja je u pytorch-u definisana kao:

`ℓ(x, y) = L = {l1, …, lN}^⊤,`

`ln= −[yn ⋅ logxn + (1 − yn) ⋅ log(1−xn)]`

Ova funkcija nudi kalkulaciju obe logaritamske komponente (npr. log(D(x)) i log(1 - D(G(z))) ).
Specifiraćemo koji deo BCE jednačine ćemo koristiti sa y ulazom. Ovo će biti izvršeno kroz petlju treniranja. Izmenom y parametra,, biramo koju komponentu želimo da izračunamo.

Dalje, želimo da definišemo labelu za prave slike sa 1, dok za izgenerisane želimo da koristimo 0.
Ove labele će biti korišćene pri kalkulaciji gubitka D i G, a takođe je ovo i konvencija u originalnom GAN istraživanju.
Na kraju, definišemo dva odvojena optimizatora, za D i G po jedan. Kao što je navedeno u DCGAN istraživanju, oba će koristiti optimizator Adam sa parametrima:

```
learning rate - 0.0002 i
beta1 - 0.5
```

Za čuvanje progresije učenja, generisaćemo fiksni skup latentnih vektora koji se iscrtavaju iz Gaussian distribucije (npr. **fixed_noise**).
U petlji treniranja, periodično ćemo unositi ovaj fixed_noise u G, i tokom iteracija ćemo videti fotografije generisane iz ove buke.

In [ ]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Treniranje

Konačno, imamo sve delove GAN framework-a definisane, sada može da se pređe na treniranje modela.
Vrlo je bitno paziti na hiperparametre, jer pogrešna vrednost za njihova podešavanja mogu dovesti do pucanja programa bez ikakvog objašnjena.

Ovde ćemo pažljivo pramitit Algoritam 1 iz Goodfellow-ovog istraživanja, dok pratimo najbolje prakse prikazane na ovom [repozitorijumu](https://github.com/soumith/ganhacks).
Konstruisaćemo različite mini skupove za prave i lažne fotografije, kao i podešavati G izlaznu funkciju da se maksimizuje logD(G(z)).

Treniranje je podeljeno u dva glavna koraka. 
Korak 1 - ažurira diskriminator,
Korak 2 - ažurira generator

* **Korak 1 - Treniranje diskriminatora**

Cilj treniranja diskriminatora jeste da maksimizuje verovatnoću da pogodi klasifikaciju fotografije da li je prava ili lažna.
Želimo da ažuriramo diskriminator tako što ćemo podići njegov stohastički gradijent (Goodfellow).
Drugim rečima, želimo da maksimizujemo

```
log(D(x)) + log(1 − D(G(z)))
```

Računaćemo ovo u dva koraka, zbog sugestija sa ganhacks repozitorijuma - separacija mini skupova.
Prvo, konstruisaćemo skup uzoraka pravih fotografija iz trening seta, proslediti ih kroz D, izračunati gubitak (**log(D(x))**), zatimm iskalkulisati gradijent u prolasku unazad.
Drugo, konstruisaćemo skup lažnih fotografija u generatoru, proslediti ovaj skup kroz D, izračunati gubitak (**log(1 - D(G(z)))**), i akumuliraćemo gradijente kroz prolaske unazad.
Kada su gradijenti prikupljeni iz oba koraka (iz skupova pravih i lažnih fotografija), optimizovaćemo diskriminator.

* **Korak 2 - Treniranje generatora**

Kao što je navedeno u originalnom istraživanju, želimo da treniramo G tako što ćemo minimizovati log(1 - D(G(z))) u pokušaju da generišemo bolje lažne fotografije.
Goodfellow je ukazao na to da ova metoda ne daje dobre rezultate (gradijente), pogotovo rano u procesu učenja.
Kao rešenje, umesto toga ćemo hteti da maksimizujemo log(D(G(z))).
U kodu ovo će se sprovesti kroz klasifikaciju izlaza iz generatora iz prvog koraka u diskriminatoru, kalkulisanje gubitka G, kalkulisanje gradijenta G u prolasku unazad i konačno ažuriranje parametara G u koraku optimizacije.

Koristićemo prave labele za funkciju gubitka, i to će nam omogućiiti da koristimo log(x) deo funkcije BCELoss, umesto log(1-x).

Na kraju, kada se završi svaka epoha, guraćemo naš skup fixed_noise u generator da vizuelno pratimo progresiju treniranja G.
Statistike treniranja koje ćemo prikazati su:

* **Loss_D** - gubitak diskriminatora koji se izračunava kao suma svih gubitaka za skupove pravih i lažnih fotografija.

`(log(D(x))+log(D(G(z)))`

* **Loss_G** - gubitak generatora izračunat kao

`log(D(G(z)))`

* **D(x)** - prosečan izlaz (u skupu) iz diskriminatora za skup pravih fotografija. Vrednost će početi blizu 1 i teoretski bi konvergiralo vrednosti 0.5 kada G postane bolji.

* **D(G(z))** - prosečan izlaz za skup lažnih slika. Prva vrednost je pre nego što je D ažuriran, a druga vrednost nakon ažuriranja D.
Vrednost će početi blizu 0 i konvergiraće ka 0.5 kada G postane bolji.

In [ ]:
# Lista za praćenje progresa
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# Za svaku epohu
for epoch in range(num_epochs):
    # Za svaki skup u dataloaderu
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Treniranje sa skupom pravih fotografija
        netD.zero_grad()
        # Formatiranje skupa
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, device=device, dtype=torch.float)
        # Prosleđivanje skupa pravih fotografija kroz D
        output = netD(real_cpu).view(-1)
        # Izračunati gubitak na skupu pravih fotografija
        errD_real = criterion(output, label)
        # Izračunati gradijente D u prolasku unazad
        errD_real.backward()
        D_x = output.mean().item()

        ## Treniranje sa skupom lažnih fotografija
        # Generisanje skupa latentnih vektora
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generisanje skupa lažnih slika pomoću G
        fake = netG(noise)
        label.fill_(fake_label)
        # Klasifikacija skupa lažnih slika pomoću D
        output = netD(fake.detach()).view(-1)
        # Izračunati gubitak D u skupu lažnih slika
        errD_fake = criterion(output, label)
        # Izračunati gradijente D u skupu lažnih slika
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Sumiranje gradijenata iz skupova pravih i lažnih slika
        errD = errD_real + errD_fake
        # Ažuriranje D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label) # labela za lažne fotografije će biti labela pravih fotografija za generator
        # Zbog ažuriranja D, izvrši prolazak unapred za skup lažnih slika kroz D
        output = netD(fake).view(-1)
        # Izračunati gubitak G baziran na ovom izlazu
        errG = criterion(output, label)
        # Izračunati gradijente G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Ažuriranje G
        optimizerG.step()

        # Statistika izlaza treniranja
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Čuvanje gubitka za plotovanje kasnije
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Provera generatora čuvanjem izlaza G uz fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Izgenerisane fotografije

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)))] for i in img_list]

In [ ]:
if not os.path.exists('../output_images'):
    os.mkdir('../output_images')
im_batch_size = 50
n_images=10000
for i_batch in range(0, n_images, im_batch_size):
    gen_z = torch.randn(im_batch_size, 100, 1, 1, device=device)
    gen_images = netG(gen_z)
    images = gen_images.to("cpu").clone().detach()
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join('../output_images', f'image_{i_batch+i_image:05d}.png'))


import shutil
shutil.make_archive('images', 'zip', '../output_images')
print("Done")

Napravljeni su rezultati tokom korišćenja 100 i 200 epoha, i njihovim poređenjem može se videti napredak u učenju ovog modela.
Sa brojem od 300 epoha, collab je izbacivao grešku preopterećenja resursa, jer ovo treniranje zahteva dosta vremena.
Rezultati su okačeni na github repozitorijumu.

# Reference

* Zasnovan na pytorch DCGAN tutorijalu [pytorch documentation - DCGAN](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

* Najbolje prakse - [ganhacks](https://github.com/soumith/ganhacks)

* [DCGAN istraživanje](https://arxiv.org/pdf/1511.06434v2.pdf)

* Kod preuzet sa [kaggle notebook](https://www.kaggle.com/rajwardhanshinde/simple-dcgan-pytorch/execution)

# Korisni linkovi:

* [Dataset Stanford Dog Images](http://vision.stanford.edu/aditya86/ImageNetDogs/)

* [Youtube GAN Computerphile](https://www.youtube.com/watch?v=Sw9r8CL98N0)

* [Youtube GAN Stanford University](https://www.youtube.com/watch?v=5WoItGTWV54)

* [wget sa drive-a](https://gist.github.com/iamtekeste/3cdfd0366ebfd2c0d805)

* [učitavanje dataseta i transformacije](https://www.kaggle.com/bitthal/understanding-input-data-and-loading-with-pytorch)

* [tehnike za optimizovanje GAN-ova](https://www.kaggle.com/jadeblue/dcgans-and-techniques-to-optimize-them)